In [5]:
# Make sure psycogp2 is installed using anaconda
# !pip install psycopg2

In [1]:
#Imports
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import matplotlib.pyplot as plt
import pandas as pd
from config import pw, username
import datetime
from datetime import datetime
import numpy 
import os

### Read the data from csv files

In [2]:
#---------------------------------------------
#Updated from config:
#---------------------------------------------
user=username
password=pw
#---------------------------------------------
#Make sure this is the name or your database in postgres
db="Beer_DB"
#Set the domin to localhoost
host="localhost"

In [3]:
#Get the path reference to read teh data

try:
    __thisdir__ = os.path.dirname(os.path.abspath(__file__))
except NameError as error:
    __thisdir__ = "."
    
#create the direcotry to store the data if not existed
review_data = os.path.join(__thisdir__, "beer_reviews.csv")
styles_data = os.path.join(__thisdir__,"JSON","data","beer_styles.csv")


In [4]:
#Reading the data
review_df = pd.read_csv(review_data)
style_df = pd.read_csv(styles_data)

In [5]:
#Check the data from the reviews
review_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
#Check the data from the reviews
style_df.head()

,id,name,beer_style,ibu_Min,ibu_Max,abv_Min,abv_Max,srm_Min,srm_Max,og_Min,fg_Min,fg_Max
0,0,Classic English-Style Pale Ale,English Pale,20,40,4.5,5.5,5,5,1.040,1.008,1.016
1,1,English-Style India Pale Ale,English IPA,35,63,5.0,7.0,6,14,1.050,1.012,1.018
2,2,Ordinary Bitter,Bitter,20,35,3.0,4.1,5,12,1.033,1.006,1.012
3,3,Special Bitter or Best Bitter,Special Bitter,28,40,4.1,4.8,6,14,1.038,1.006,1.012
4,4,Extra Special Bitter,ESB,30,45,4.8,5.8,8,14,1.046,1.010,1.016


### Normalize the data

In [7]:
#Normilize the data
review_style_done=list()

#Key for new entries
new_key = style_list = list()

#We need to iterate all beer styles find all matches in teh review and normilize the data
for style in style_df["beer_style"]:
        style_list.append(style)
        #Find all dat athat matches the style defined
        matches_df = pd.DataFrame(review_df["beer_style"]\
                                 [review_df.beer_style.str.contains(style, regex= True, na=False)])
        #Remove the duplicates
        matches_df.drop_duplicates(inplace=True)
        #Iterate all matches and replace the data
        for match in matches_df["beer_style"]:
            #Replace the data if not updated yet and mask it as updated
            if match not in review_style_done:
                print(f"{match} : {style}")
                if match != style:
                    review_df.replace({'beer_style':{match:style}},inplace=True)
                review_style_done.append(match)

English Pale Ale : English Pale
English Pale Mild Ale : English Pale
English Bitter : Bitter
Extra Special / Strong Bitter (ESB) : Bitter
English Dark Mild Ale : English Dark Mild
English Brown Ale : English Brown
Old Ale : Old Ale
English Strong Ale : Strong Ale
American Strong Ale : Strong Ale
Scotch Ale / Wee Heavy : Scotch Ale
Milk / Sweet Stout : Sweet Stout
Oatmeal Stout : Oatmeal Stout
Irish Red Ale : Irish Red
Foreign / Export Stout : Export Stout
American Pale Lager : American Pale
American Pale Ale (APA) : American Pale
American Pale Wheat Ale : American Pale
American IPA : American IPA
American Double / Imperial IPA : Imperial IPA
American Amber / Red Ale : Amber
American Amber / Red Lager : Amber
American Barleywine : American Barleywine
Wheatwine : Wheatwine
American Blonde Ale : Blonde
American Brown Ale : American Brown
American Black Ale : Black Ale
American Stout : American Stout
Export Stout : Stout
Russian Imperial Stout : Stout
American Double / Imperial Stout : Sto

### Build the tables

In [14]:
#--------------------------------------#
#            Beer Styles Table         #
#--------------------------------------#
import numpy as np
#Next key for styles
key_id = len(style_df)
#To creat enew entries
#createing list for each column,
#this is faster than append rows to teh data frame
id = list()
name = list()
short_name = list()
ibuMin = list()
ibuMax = list()
abvMin = list()
abvMax = list()
srmMin = list()
srmMax = list()
ogMin = list()
fgMin = list()
fgMax = list()
#After data normalization get all the styles witouth record and create a new entry
normal_styles_df = pd.DataFrame(review_df["beer_style"])
#Remove the duplicates
normal_styles_df.drop_duplicates(inplace=True)
normal_styles_df
#Get all data syles not defined yet
for style in normal_styles_df["beer_style"]:
   if style not in style_list:
       #not_defined.append(style)
       name.append(style)
       short_name.append(style)
       id.append(key_id)
       #Since this styles are not defined set them to na
       ibuMin.append(np.NaN)
       ibuMax.append(np.NaN)
       abvMin.append(np.NaN)
       abvMax.append(np.NaN)
       srmMin.append(np.NaN)
       srmMax.append(np.NaN)
       ogMin.append(np.NaN)
       fgMin.append(np.NaN)
       fgMax.append(np.NaN)
       key_id+=1
#Build the new data frame
ndef_df = pd.DataFrame({"id":id,
                       "name":name,
                       "beer_style":short_name,
                       "ibu_Min":ibuMin,
                       "ibu_Max":ibuMax,
                       "abv_Min":abvMin,
                       "abv_Max":abvMax,
                       "srm_Min":srmMin,
                       "srm_Max":srmMax,
                       "og_Min":ogMin,
                       "fg_Min":fgMin,
                       "fg_Max":fgMax})
#Join the data
beer_styles_df = pd.concat([style_df, ndef_df])
beer_styles_df.rename(columns={"id":"beer_style_id"},inplace=True)
beer_styles_df.rename(columns={"name":"beer_new_name"},inplace=True)
beer_styles_df.head()

,beer_style_id,beer_new_name,beer_style,ibu_Min,ibu_Max,abv_Min,abv_Max,srm_Min,srm_Max,og_Min,fg_Min,fg_Max
0,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,1.008,1.016
1,1,English-Style India Pale Ale,English IPA,35.0,63.0,5.0,7.0,6.0,14.0,1.050,1.012,1.018
2,2,Ordinary Bitter,Bitter,20.0,35.0,3.0,4.1,5.0,12.0,1.033,1.006,1.012
3,3,Special Bitter or Best Bitter,Special Bitter,28.0,40.0,4.1,4.8,6.0,14.0,1.038,1.006,1.012
4,4,Extra Special Bitter,ESB,30.0,45.0,4.8,5.8,8.0,14.0,1.046,1.010,1.016


In [17]:
new_merge = pd.merge(beer_styles_df,review_df,how="outer")

In [18]:
new_merge.dropna(inplace=True)

In [19]:
new_merge

,beer_style_id,beer_new_name,beer_style,ibu_Min,ibu_Max,abv_Min,abv_Max,srm_Min,srm_Max,og_Min,...,review_time,review_overall,review_aroma,review_appearance,review_profilename,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.294325e+09,5.0,4.5,4.5,JamesS,5.0,4.0,Lawn Mower Pale Ale,5.0,61478.0
1,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.283115e+09,4.5,4.0,4.0,brundo4,4.0,3.5,Lawn Mower Pale Ale,5.0,61478.0
4,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.130157e+09,4.5,3.5,4.0,oberon,3.5,4.0,Moon Dance Pale Ale,4.2,26562.0
5,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.314904e+09,3.5,3.5,3.5,smcolw,4.0,4.0,Skyelight,3.8,71157.0
6,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.311007e+09,3.5,3.0,3.0,AgentMunky,4.0,4.0,Skyelight,3.8,71157.0
7,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.314822e+09,2.0,2.0,3.5,smcolw,3.5,1.5,Red Cuillin,4.2,11456.0
8,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.283200e+09,3.5,3.5,4.0,champ103,3.5,3.0,Red Cuillin,4.2,11456.0
9,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.276450e+09,4.0,4.5,4.0,dqrull,4.5,4.0,Red Cuillin,4.2,11456.0
10,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.258378e+09,3.0,3.0,3.5,mrmanning,3.0,3.5,Red Cuillin,4.2,11456.0
11,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,...,1.238361e+09,4.0,4.0,3.5,mduncan,3.5,4.0,Red Cuillin,4.2,11456.0


In [20]:
new_merge.to_csv("less_big.csv")